In [ ]:
!pip install pandas numpy scikit-learn joblib streamlit plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
from datetime import datetime, timedelta

In [ ]:
# Generate synthetic traffic data
def generate_traffic_data(n_samples=1000):
    np.random.seed(42)

    # Generate timestamps
    base_date = datetime(2024, 1, 1)
    timestamps = [base_date + timedelta(hours=i) for i in range(n_samples)]

    data = {
        'timestamp': timestamps,
        'day_of_week': [ts.weekday() for ts in timestamps],
        'hour': [ts.hour for ts in timestamps],
        'temperature': np.random.uniform(0, 35, n_samples),
        'precipitation': np.random.uniform(0, 50, n_samples),
        'special_event': np.random.choice([0, 1], n_samples, p=[0.9, 0.1]),
        'road_work': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
        'vehicle_count': np.random.poisson(1000, n_samples),
    }

    df = pd.DataFrame(data)

    # Create realistic congestion levels
    base_congestion = (
        5 * np.sin(df['hour'] * np.pi / 12) +
        3 * (df['day_of_week'] < 5) +
        2 * ((df['hour'] >= 8) & (df['hour'] <= 10)) +
        2 * ((df['hour'] >= 16) & (df['hour'] <= 18)) +
        0.05 * df['temperature'] +
        0.1 * df['precipitation'] +
        3 * df['special_event'] +
        2 * df['road_work'] +
        0.01 * (df['vehicle_count'] - 1000) +
        np.random.normal(0, 0.5, n_samples)
    )

    df['congestion_level'] = np.clip(base_congestion, 0, 10)
    return df

In [ ]:
# Feature engineering
def prepare_features(df):
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

    features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos',
                'temperature', 'precipitation', 'special_event',
                'road_work', 'vehicle_count']

    return df[features]

In [ ]:
def main():
    print("Generating synthetic data...")
    df = generate_traffic_data()

    print("Preparing features...")
    X = prepare_features(df)
    y = df['congestion_level']

    print("Splitting and scaling data...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    print("Training model...")
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    print("Saving model and scaler...")
    joblib.dump(model, 'traffic_predictor_model.joblib')
    joblib.dump(scaler, 'traffic_scaler.joblib')

    print("Training complete! You can now run the Streamlit app.")

if __name__ == "__main__":
    main()

Generating synthetic data...
Preparing features...
Splitting and scaling data...
Training model...
Saving model and scaler...
Training complete! You can now run the Streamlit app.
